In [1]:
from pathlib import Path

path = Path("../../storage")

In [2]:
from climate_tutorial.data import download

In [ ]:
download(root = path, source = "weatherbench", variable = "2m_temperature", dataset = "era5", resolution = "5.625")

In [3]:
download(root = path, source = "weatherbench", variable = "geopotential", dataset = "era5", resolution = "5.625")

--2022-10-14 03:07:20--  https://dataserv.ub.tum.de/s/m1524895/download?path=%2F5.625deg%2Fgeopotential&files=geopotential_5.625deg.zip
Resolving dataserv.ub.tum.de (dataserv.ub.tum.de)... 138.246.224.34, 2001:4ca0:800::8af6:e022
Connecting to dataserv.ub.tum.de (dataserv.ub.tum.de)|138.246.224.34|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 28509844682 (27G) [application/zip]
Saving to: ‘../../storage/data/weatherbench/era5/5.625/geopotential.zip’

     0K .......... .......... .......... .......... ..........  0%  153K 2d2h
    50K .......... .......... .......... .......... ..........  0%  306K 37h55m
   100K .......... .......... .......... .......... ..........  0%  163M 25h17m
   150K .......... .......... .......... .......... ..........  0% 7.44M 19h13m
   200K .......... .......... .......... .......... ..........  0%  317K 20h15m
   250K .......... .......... .......... .......... ..........  0% 7

  5000K .......... .......... .......... .......... ..........  0%  282K 2h23m
  5050K .......... .......... .......... .......... ..........  0% 23.8M 2h22m
  5100K .......... .......... .......... .......... ..........  0% 24.8M 2h21m
  5150K .......... .......... .......... .......... ..........  0% 24.4M 2h20m
  5200K .......... .......... .......... .......... ..........  0% 21.0M 2h19m
  5250K .......... .......... .......... .......... ..........  0% 24.9M 2h18m
  5300K .......... .......... .......... .......... ..........  0% 24.6M 2h16m
  5350K .......... .......... .......... .......... ..........  0% 25.8M 2h15m
  5400K .......... .......... .......... .......... ..........  0% 21.8M 2h14m
  5450K .......... .......... .......... .......... ..........  0% 12.8M 2h13m
  5500K .......... .......... .......... .......... ..........  0% 20.4M 2h12m
  5550K .......... .......... .......... .......... ..........  0% 6.70M 2h12m
  5600K .......... .......... .......... .......... 

 11150K .......... .......... .......... .......... ..........  0%  157K 1h52m
 11200K .......... .......... .......... .......... ..........  0% 22.6M 1h51m
 11250K .......... .......... .......... .......... ..........  0% 24.7M 1h51m
 11300K .......... .......... .......... .......... ..........  0% 12.0M 1h50m
 11350K .......... .......... .......... .......... ..........  0% 25.9M 1h50m
 11400K .......... .......... .......... .......... ..........  0% 23.0M 1h50m
 11450K .......... .......... .......... .......... ..........  0% 26.3M 1h49m
 11500K .......... .......... .......... .......... ..........  0% 26.5M 1h49m
 11550K .......... .......... .......... .......... ..........  0% 24.6M 1h48m
 11600K .......... .......... .......... .......... ..........  0% 22.3M 1h48m
 11650K .......... .......... .......... .......... ..........  0% 27.8M 1h48m
 11700K .......... .......... .......... .......... ..........  0% 27.4M 1h47m
 11750K .......... .......... .......... .......... 

 17300K .......... .......... .......... .......... ..........  0%  380K 89m18s
 17350K .......... .......... .......... .......... ..........  0% 25.0M 89m6s
 17400K .......... .......... .......... .......... ..........  0% 10.4M 88m58s
 17450K .......... .......... .......... .......... ..........  0% 18.2M 88m47s
 17500K .......... .......... .......... .......... ..........  0% 25.3M 88m35s
 17550K .......... .......... .......... .......... ..........  0% 25.0M 88m23s
 17600K .......... .......... .......... .......... ..........  0% 19.0M 88m12s
 17650K .......... .......... .......... .......... ..........  0% 19.6M 88m1s
 17700K .......... .......... .......... .......... ..........  0% 24.4M 87m49s
 17750K .......... .......... .......... .......... ..........  0% 21.2M 87m38s
 17800K .......... .......... .......... .......... ..........  0% 14.6M 87m28s
 17850K .......... .......... .......... .......... ..........  0% 25.1M 87m17s
 17900K .......... .......... .......... .

 23450K .......... .......... .......... .......... ..........  0%  228K 86m47s
 23500K .......... .......... .......... .......... ..........  0% 10.5M 86m42s
 23550K .......... .......... .......... .......... ..........  0% 17.6M 86m34s
 23600K .......... .......... .......... .......... ..........  0% 21.7M 86m25s
 23650K .......... .......... .......... .......... ..........  0% 25.6M 86m17s
 23700K .......... .......... .......... .......... ..........  0% 26.1M 86m8s
 23750K .......... .......... .......... .......... ..........  0% 18.3M 86m0s
 23800K .......... .......... .......... .......... ..........  0% 22.6M 85m52s
 23850K .......... .......... .......... .......... ..........  0% 22.8M 85m44s
 23900K .......... .......... .......... .......... ..........  0% 27.3M 85m35s
 23950K .......... .......... .......... .......... ..........  0% 25.4M 85m27s
 24000K .......... .......... .......... .......... ..........  0% 10.6M 85m21s
 24050K .......... .......... .......... .

...

121050K .......... .......... .......... .......... ..........  0% 24.7M 84m49s
121100K .......... .......... .......... .......... ..........  0% 25.9M 84m47s
121150K .......... .......... .......... .......... ..........  0% 13.1M 84m46s
121200K .......... .......... .......... .......... ..........  0% 12.9M 84m44s
121250K .......... .......... .......... .......... ..........  0% 24.3M 84m43s
121300K .......... .......... .......... .......... ..........  0% 16.7M 84m41s
121350K .......... .......... .......... .......... ..........  0% 23.9M 84m40s
121400K .......... .......... .......... .......... ..........  0% 21.7M 84m38s
121450K .......... .......... .......... .......... ..........  0% 25.1M 84m37s
121500K .......... .......... .......... .......... ..........  0% 24.6M 84m35s
121550K .......... .......... .......... .......... ..........  0% 6.92M 84m34s
121600K .......... .......... .......... .......... ..........  0% 4.25M 84m35s
121650K .......... .......... ..........

126850K .......... .......... .......... .......... ..........  0%  170K 85m39s
126900K .......... .......... .......... .......... ..........  0% 25.9M 85m37s
126950K .......... .......... .......... .......... ..........  0% 25.8M 85m36s
127000K .......... .......... .......... .......... ..........  0% 10.7M 85m35s
127050K .......... .......... .......... .......... ..........  0% 5.09M 85m35s
127100K .......... .......... .......... .......... ..........  0% 27.2M 85m33s
127150K .......... .......... .......... .......... ..........  0% 25.4M 85m31s
127200K .......... .......... .......... .......... ..........  0% 18.9M 85m30s
127250K .......... .......... .......... .......... ..........  0% 24.7M 85m28s
127300K .......... .......... .......... .......... ..........  0% 28.4M 85m27s
127350K .......... .......... .......... .......... ..........  0% 24.9M 85m25s
127400K .......... .......... .......... .......... ..........  0% 22.6M 85m24s
127450K .......... .......... ..........

131950K .......... .......... .......... .......... ..........  0% 8.18M 84m51s
132000K .......... .......... .......... .......... ..........  0%  273K 85m27s
132050K .......... .......... .......... .......... ..........  0% 23.3M 85m26s
132100K .......... .......... .......... .......... ..........  0% 23.3M 85m24s
132150K .......... .......... .......... .......... ..........  0% 9.08M 85m24s
132200K .......... .......... .......... .......... ..........  0% 18.0M 85m22s
132250K .......... .......... .......... .......... ..........  0% 25.1M 85m21s
132300K .......... .......... .......... .......... ..........  0% 15.2M 85m19s
132350K .......... .......... .......... .......... ..........  0% 25.0M 85m18s
132400K .......... .......... .......... .......... ..........  0% 20.7M 85m16s
132450K .......... .......... .......... .......... ..........  0% 26.4M 85m15s
132500K .......... .......... .......... .......... ..........  0% 26.0M 85m13s
132550K .......... .......... ..........

138100K .......... .......... .......... .......... ..........  0%  526K 86m20s
138150K .......... .......... .......... .......... ..........  0% 15.2M 86m19s
138200K .......... .......... .......... .......... ..........  0% 17.7M 86m17s
138250K .......... .......... .......... .......... ..........  0% 20.6M 86m16s
138300K .......... .......... .......... .......... ..........  0% 18.0M 86m14s
138350K .......... .......... .......... .......... ..........  0% 11.3M 86m13s
138400K .......... .......... .......... .......... ..........  0% 13.9M 86m12s
138450K .......... .......... .......... .......... ..........  0% 21.6M 86m11s
138500K .......... .......... .......... .......... ..........  0% 22.7M 86m9s
138550K .......... .......... .......... .......... ..........  0% 20.6M 86m8s
138600K .......... .......... .......... .......... ..........  0% 18.5M 86m7s
138650K .......... .......... .......... .......... ..........  0% 22.6M 86m5s
138700K .......... .......... .......... ...

KeyboardInterrupt: 

In [ ]:
path = path/"data/weatherbench/era5/5.625"

In [ ]:
from climate_tutorial.utils.data import load_dataset, view

t_dataset = load_dataset(path/"2m_temperature")
z_dataset = load_dataset(path/"geopotential")
view(t_dataset)

In [ ]:
from climate_tutorial.utils.datetime import Year, Days, Hours
from climate_tutorial.data import DataModule

data_module = DataModule(
    dataset = "ERA5",
    task = "forecasting",
    root_dir = path,
    in_vars = ["2m_temperature", "geopotential"],
    out_vars = ["2m_temperature", "geopotential"],
    train_start_year = Year(1979),
    val_start_year = Year(2015),
    test_start_year = Year(2017),
    end_year = Year(2018),
    pred_range = Days(3),
    subsample = Hours(6),
    batch_size = 128,
    num_workers = 1
)

In [ ]:
from climate_tutorial.models import load_model

model_kwargs = {
    "img_size": [32, 64],
    "patch_size": 2,
    "drop_path": 0.1,
    "drop_rate": 0.1,
    "learn_pos_emb": True,
    "in_vars": data_module.hparams.in_vars,
    "out_vars": data_module.hparams.out_vars,
    "embed_dim": 128,
    "depth": 8,
    "decoder_depth": 0,
    "num_heads": 4,
    "mlp_ratio": 4,
}

optim_kwargs = {
    "lr": 1e-4,
    "weight_decay": 1e-5,
    "warmup_epochs": 1,
    "max_epochs": 5,
}

model_module = load_model(name = "vit", task = "forecasting", model_kwargs = model_kwargs, optim_kwargs = optim_kwargs)
# model_module = load_model(name = "resnet", task = "forecasting", model_kwargs = model_kwargs, optim_kwargs = optim_kwargs)
# # model_module = load_model(name = "unet", task = "forecasting", model_kwargs = model_kwargs, optim_kwargs = optim_kwargs)

In [ ]:
# add_description
from climate_tutorial.models import set_climatology
set_climatology(model_module, data_module)

In [ ]:
from climate_tutorial.training import Trainer, WandbLogger

trainer = Trainer(
    seed = 0,
    accelerator = "gpu",
    precision = 16,
    max_epochs = 5,
    # logger = WandbLogger(project = "climate_tutorial", name = "forecast-vit")
)

In [ ]:
trainer.fit(model_module, data_module)

In [ ]:
trainer.test(model_module, data_module)